# Data Preprocessing On Different Datasets

This notebook is used to preprocess the data for the different datasets. Each dataset has a different issue, preprocessing method. The preprocessing methods are described in the following sections.

## 1. Handling Missing Values

### Dataset Context

[San Francisco Building Permits](https://www.kaggle.com/datasets/aparnashastry/building-permit-applications-data)
A building permit is an official approval document issued by a governmental agency that allows you or your contractor to proceed with a construction or remodeling project on one's property. For more details go to https://www.thespruce.com/what-is-a-building-permit-1398344. Each city or county has its own office related to buildings, that can do multiple functions like issuing permits, inspecting buildings to enforce safety measures, modifying rules to accommodate needs of the growing population etc. For the city of San Francisco, permit issuing is taken care by www.sfdbi.org/

Why is this important: In the recent past, several posts and blogs highlighted that main discrepancy in demand and supply in real estate industry is due to delays in issuing building permits. Refer:
https://www.trulia.com/blog/trends/elasticity-2016/ - Introduces concept of elasticity, and nice scatterplot of various cities. A good data story!
https://biv.com/article/2014/11/city-building-permit-delays-costing-developers-tim

### Take a first look at the data

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../input/building-permit-applications-data/Building_Permits.csv')
df.head(5)

C:\Users\duongdinh\AppData\Local\Temp\ipykernel_17068\1935086651.py:1: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../input/building-permit-applications-data/Building_Permits.csv')


,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,NaN,Ellis,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,NaN,Geary,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,NaN,Pacific,Av,...,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,NaN,Pacific,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,NaN,Market,St,...,3.0,constr type 3,NaN,NaN,NaN,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198900 entries, 0 to 198899
Data columns (total 43 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   Permit Number                           198900 non-null  object 
 1   Permit Type                             198900 non-null  int64  
 2   Permit Type Definition                  198900 non-null  object 
 3   Permit Creation Date                    198900 non-null  object 
 4   Block                                   198900 non-null  object 
 5   Lot                                     198900 non-null  object 
 6   Street Number                           198900 non-null  int64  
 7   Street Number Suffix                    2216 non-null    object 
 8   Street Name                             198900 non-null  object 
 9   Street Suffix                           196132 non-null  object 
 10  Unit                                    2947

In [4]:
df.describe()

,Permit Type,Street Number,Unit,Number of Existing Stories,Number of Proposed Stories,Estimated Cost,Revised Cost,Existing Units,Proposed Units,Plansets,Existing Construction Type,Proposed Construction Type,Supervisor District,Zipcode,Record ID
count,198900.000000,198900.000000,29479.000000,156116.000000,156032.000000,1.608340e+05,1.928340e+05,147362.000000,147989.000000,161591.000000,155534.000000,155738.000000,197183.000000,197184.000000,1.989000e+05
mean,7.522323,1121.728944,78.517182,5.705773,5.745043,1.689554e+05,1.328562e+05,15.666164,16.510950,1.274650,4.072878,4.089529,5.538403,94115.500558,1.162048e+12
std,1.457451,1135.768948,326.981324,8.613455,8.613284,3.630386e+06,3.584903e+06,74.476321,75.220444,22.407345,1.585756,1.578766,2.887041,9.270131,4.918215e+11
min,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,94102.000000,1.293532e+10
25%,8.000000,235.000000,0.000000,2.000000,2.000000,3.300000e+03,1.000000e+00,1.000000,1.000000,0.000000,3.000000,3.000000,3.000000,94109.000000,1.308567e+12
50%,8.000000,710.000000,0.000000,3.000000,3.000000,1.100000e+04,7.000000e+03,1.000000,2.000000,2.000000,5.000000,5.000000,6.000000,94114.000000,1.371840e+12
75%,8.000000,1700.000000,1.000000,4.000000,4.000000,3.500000e+04,2.870750e+04,4.000000,4.000000,2.000000,5.000000,5.000000,8.000000,94122.000000,1.435000e+12
max,8.000000,8400.000000,6004.000000,78.000000,78.000000,5.379586e+08,7.805000e+08,1907.000000,1911.000000,9000.000000,5.000000,5.000000,11.000000,94158.000000,1.498342e+12


This table has many empty values. We need to handle them.

In [5]:
missing_values_count = df.isnull().sum()

missing_values_count

Permit Number                                  0
Permit Type                                    0
Permit Type Definition                         0
Permit Creation Date                           0
Block                                          0
Lot                                            0
Street Number                                  0
Street Number Suffix                      196684
Street Name                                    0
Street Suffix                               2768
Unit                                      169421
Unit Suffix                               196939
Description                                  290
Current Status                                 0
Current Status Date                            0
Filed Date                                     0
Issued Date                                14940
Completed Date                            101709
First Construction Document Date           14946
Structural Notification                   191978
Number of Existing S

In [6]:
total_cell = np.product(df.shape)
total_missing = missing_values_count.sum()
percent_missing = (total_missing / total_cell) * 100
print("Missing percentage: ", percent_missing)

Missing percentage:  26.26002315058403


More than a quarter of the data is missing. Now, we need to look into each column and find out that they are not recorded or not existed.
- Not Existed: Street Number Suffix, Street Suffix, Unit, Unit Suffix, Description, Issued Date, Complete Date, First Construction Document Date, Structural Notification, Number of Existing Stories, Number of Proposed Stories, Voluntary Soft-Story Retrofit, Fire Only Permit, Plansets, TIDF Compliance, Existing Construction Type, Existing Construction Type , Proposed Construction Type, Proposed Construction Type Description, Supervisor District, Neighborhoods - Analysis Boundaries, Zipcode, Location
- Not Recorded: Estimated Cost, Revised Cost, Existing Use, Existing Units, Proposed Use, Proposed Units, Site Permit

All building must have a Location but I put it in the Not Existed because the number of empty Zipcode is the same so I suspect people may leave them empty for good reasons. Similar idea might apply to other columns.

### Try Drop Missing Values: Rows

In [7]:
df.dropna()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID


### Try Drop Missing Values: Columns

In [8]:
df_no_empty = df.dropna(axis=1)
df_no_empty.head(5)

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Current Status,Current Status Date,Filed Date,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,Ellis,expired,12/21/2017,05/06/2015,1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,Geary,issued,08/03/2017,04/19/2016,1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,Pacific,withdrawn,09/26/2017,05/27/2016,1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,Pacific,complete,07/24/2017,11/07/2016,1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,Market,issued,12/01/2017,11/28/2016,144548169992


In [9]:
drop_columns = df.shape[1] - df_no_empty.shape[1]
print("Drop {} columns".format(drop_columns))

Drop 31 columns


### Try Fill Missing Values

For "Not Existed" values, it makes no sense filling them. We will work on "Not Recorded" values.
For "Not Recorded" values, we will fill them with the most frequent value.

In [10]:
not_record_columns = ['Estimated Cost', 'Revised Cost', 'Existing Use', 'Existing Units', 'Proposed Use', 'Proposed Units', 'Site Permit']
for column in not_record_columns:
    df_no_empty[column] = df[column].fillna(df[column].value_counts().idxmax())

C:\Users\duongdinh\AppData\Local\Temp\ipykernel_17068\1065663124.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_empty[column] = df[column].fillna(df[column].value_counts().idxmax())
C:\Users\duongdinh\AppData\Local\Temp\ipykernel_17068\1065663124.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_empty[column] = df[column].fillna(df[column].value_counts().idxmax())
C:\Users\duongdinh\AppData\Local\Temp\ipykernel_17068\1065663124.py:3: SettingWithCopyWarning: 
A value is trying to be set on

In [11]:
df_no_empty.head(10)

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Current Status,Current Status Date,Filed Date,Record ID,Estimated Cost,Revised Cost,Existing Use,Existing Units,Proposed Use,Proposed Units,Site Permit
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,Ellis,expired,12/21/2017,05/06/2015,1380611233945,4000.0,4000.0,tourist hotel/motel,143.0,1 family dwelling,1.0,Y
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,Geary,issued,08/03/2017,04/19/2016,1420164406718,1.0,500.0,tourist hotel/motel,1.0,1 family dwelling,1.0,Y
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,Pacific,withdrawn,09/26/2017,05/27/2016,1424856504716,20000.0,1.0,retail sales,39.0,retail sales,39.0,Y
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,Pacific,complete,07/24/2017,11/07/2016,1443574295566,2000.0,2000.0,1 family dwelling,1.0,1 family dwelling,1.0,Y
4,201611283529,6,demolitions,11/28/2016,0342,001,950,Market,issued,12/01/2017,11/28/2016,144548169992,100000.0,100000.0,retail sales,1.0,1 family dwelling,1.0,Y
5,201706149344,8,otc alterations permit,06/14/2017,4105,009,800,Indiana,issued,07/06/2017,06/14/2017,1466911170855,4000.0,4000.0,apartments,326.0,apartments,326.0,Y
6,201706300814,8,otc alterations permit,06/30/2017,1739,020,1291,11th,complete,07/12/2017,06/30/2017,1468970403692,12000.0,12000.0,apartments,5.0,apartments,5.0,Y
7,M803667,8,otc alterations permit,06/30/2017,4789,014,1465,Revere,issued,06/30/2017,06/30/2017,1469035175050,1.0,0.0,1 family dwelling,1.0,1 family dwelling,1.0,Y
8,M804227,8,otc alterations permit,07/05/2017,1212,054,2094,Fell,issued,07/05/2017,07/05/2017,1469198253772,1.0,1.0,1 family dwelling,1.0,1 family dwelling,1.0,Y
9,M804767,8,otc alterations permit,07/06/2017,1259,016,89,Alpine,issued,07/06/2017,07/06/2017,146932394171,1.0,0.0,1 family dwelling,1.0,1 family dwelling,1.0,Y


In [12]:
df_no_empty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198900 entries, 0 to 198899
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Permit Number           198900 non-null  object 
 1   Permit Type             198900 non-null  int64  
 2   Permit Type Definition  198900 non-null  object 
 3   Permit Creation Date    198900 non-null  object 
 4   Block                   198900 non-null  object 
 5   Lot                     198900 non-null  object 
 6   Street Number           198900 non-null  int64  
 7   Street Name             198900 non-null  object 
 8   Current Status          198900 non-null  object 
 9   Current Status Date     198900 non-null  object 
 10  Filed Date              198900 non-null  object 
 11  Record ID               198900 non-null  int64  
 12  Estimated Cost          198900 non-null  float64
 13  Revised Cost            198900 non-null  float64
 14  Existing Use        